In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# The goal of preprocessing this DataSet is to convert it into a joint dataset. The joint dataset aims to present a better functionality 
# by presenting its contents, such that they are arranged in a feasible form for our program.
# It contains each song with its respective genre information, release year, and audio features.

In [3]:
# We use the pandas library to combine the required datasets
import pandas as pd

In [ ]:
# Here, 'data_dir' takes the path of the folder which contains the Data Sources files
data_dir = 'Desktop/Engage_22/Experiment-1/Saturday/SpotGenTrack/Data Sources'
albums_data= pd.read_csv(data_dir + 'spotify_albums.csv')
artists_data= pd.read_csv(data_dir + 'spotify_artists.csv')
tracks_data= pd.read_csv(data_dir + 'spotify_tracks.csv')

In [6]:
# We display the individual datasets
display(albums_data.head())
albums_data.columns

In [ ]:
display(artists_data.head())
artists_data.columns

In [7]:
display(tracks_data.head())
tracks_data.columns

In [24]:
# Join artist genre information and album release date with track dataset
# Dropping irrelevant columns and getting only tracks after 1990 

def join_genre_and_date(artist_df, album_df, track_df):
    album= album_df.rename(columns= {'id': "album_id"}).set_index('album_id')

    artist= artist_df.rename(columns= {'id':"artists_id", 'name': "artists_name"}).set_index('artists_id')

    track= track_df.set_index('album_id').join (album['release_date'],
on='album_id' ) 

    track.artists_id= track.artists_id.apply(lambda x: x[2: -2]) 
    track = track.set_index('artists_id').join(artist[['artists_name', 'genres']], on='artists_id' )

    track.reset_index (drop=False, inplace=True)
    track['release_year']= pd.to_datetime(track.release_date).dt.year
    track.drop(columns = ['Unnamed: 0', 'country', 'track_name_prev', 'track_number', 'type'], inplace = True)
    return track[track.release_year >= 1990]

In [30]:
# Narrowing down the size of the available DataSet by including only songs from certain genres
# The datafrome returned by the 'get_filtered_track_df' belong to the genres mentioned in the 'genres_to_include'
def get_filtered_track_df(df, genres_to_include):
    df['genres'] = df.genres.apply(lambda x: [i[1:-1] for i in str(x)
                                             [1:-1].split(", ")])
    df_exploded = df.explode("genres")[df.explode("genres")
                                       ["genres"].isin(genres_to_include)]
    df_exploded.loc[df_exploded["genres"]=="korean pop", "genres"] = "kpop"
    df_exploded_indices = list(df_exploded.index.unique())
    df = df[df.index.isin(df_exploded_indices)]
    df = df.reset_index(drop=True)
    return df

In [31]:
# Final preprocessing for the uri column which will be used later
# Further irrelevant columns are dropped at this stage
genres_to_include = genres = ['dance pop', 'electronic','electropop', 'hip hop', 'jazz',
                              'k-pop', 'latin', 'pop', 'pop rap', 'r&b', 'rock']

track_with_year_and_genre= join_genre_and_date(artists_data, albums_data, tracks_data)

filtered_track_df= get_filtered_track_df(track_with_year_and_genre, genres_to_include)

In [26]:
# Checking for the pandas version as it prompts errors sometimes
pd.__version__

In [33]:
filtered_track_df["uri"]= filtered_track_df["uri"].str.replace("spotify:track:", "")
filtered_track_df= filtered_track_df.drop(columns= ['analysis_url', 'available_markets'])

In [34]:
display(filtered_track_df.head())
filtered_track_df.columns

In [ ]:
# Finally saving the data into a file of our requirement
filtered_track_df.to_csv("filtered_track_df.csv", index=False)